# 線形代数：行列の内積（Matrix Inner Product）

このノートブックでは、**行列の内積（matrix inner product）**について基礎から学びます。

## 目次
1. [行列の内積とは](#行列の内積とは)
2. [Frobenius内積](#Frobenius内積)
3. [数学的な定義](#数学的な定義)
4. [行列の内積の性質](#行列の内積の性質)
5. [実装例](#実装例)
6. [ベクトル内積との関係](#ベクトル内積との関係)
7. [機械学習での応用例](#機械学習での応用例)


---

## 必要なライブラリのインポート


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

# 日本語フォントの設定
try:
    jp_fonts = ['Hiragino Sans', 'Hiragino Kaku Gothic ProN', 'Yu Gothic',
                'Noto Sans CJK JP', 'AppleGothic', 'Arial Unicode MS']
    available_fonts = [f.name for f in fm.fontManager.ttflist]

    font_found = False
    for font_name in jp_fonts:
        if font_name in available_fonts:
            plt.rcParams['font.family'] = font_name
            font_found = True
            print(f"Using font: {font_name}")
            break

    if not font_found:
        plt.rcParams['font.family'] = 'sans-serif'
        plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Hiragino Kaku Gothic ProN',
                                            'Yu Gothic', 'Meiryo', 'MS Gothic', 'DejaVu Sans']
        print("Using fallback font configuration")
except Exception as e:
    print(f"Font configuration warning: {e}")
    plt.rcParams['font.family'] = 'sans-serif'

plt.rcParams['mathtext.default'] = 'regular'
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



Using font: Hiragino Sans


---

## 行列の内積とは

**行列の内積（matrix inner product）**は、2つの行列から1つのスカラー値を得る演算です。ベクトルの内積を行列に拡張した概念です。

### ベクトル内積との違い

- **ベクトルの内積**: 2つのベクトル $\mathbf{a}$ と $\mathbf{b}$ からスカラー値を得る
  - $\mathbf{a} \cdot \mathbf{b} = \sum_{i} a_i b_i$
  
- **行列の内積**: 2つの行列 $A$ と $B$ からスカラー値を得る
  - 行列を「ベクトル化」してから内積を取る、またはFrobenius内積を使用

### なぜ行列の内積が重要なのか？

- **行列の類似度測定**: 2つの行列がどの程度似ているかを測定
- **最適化問題**: 機械学習の損失関数で使用
- **ノルムの定義**: 行列のFrobeniusノルムの定義に使用
- **機械学習**: ニューラルネットワークの重み行列の比較など


---

## Frobenius内積

行列の内積として最も一般的なのは**Frobenius内積（Frobenius inner product）**です。

### 定義

2つの同じサイズの行列 $A \in \mathbb{R}^{m \times n}$ と $B \in \mathbb{R}^{m \times n}$ のFrobenius内積は：

$$\langle A, B \rangle_F = \sum_{i=1}^{m} \sum_{j=1}^{n} A_{ij} B_{ij} = \text{tr}(A^T B)$$

ここで、$\text{tr}(\cdot)$ はトレース（対角成分の和）を表します。

### 簡単な例

$$A = \begin{pmatrix} 1 & 2 \\ 3 & 4 \end{pmatrix}, \quad B = \begin{pmatrix} 5 & 6 \\ 7 & 8 \end{pmatrix}$$

$$\langle A, B \rangle_F = 1 \times 5 + 2 \times 6 + 3 \times 7 + 4 \times 8 = 5 + 12 + 21 + 32 = 70$$


In [2]:
# Frobenius内積の計算例
A = np.array([[1, 2],
              [3, 4]])
B = np.array([[5, 6],
              [7, 8]])

print("行列 A:")
print(A)
print("\n行列 B:")
print(B)

# 方法1: 要素ごとの積の和
frobenius_inner_1 = np.sum(A * B)
print(f"\n方法1: np.sum(A * B) = {frobenius_inner_1}")

# 方法2: トレースを使用 (tr(A^T B))
frobenius_inner_2 = np.trace(A.T @ B)
print(f"方法2: np.trace(A.T @ B) = {frobenius_inner_2}")

# 方法3: ベクトル化してから内積
frobenius_inner_3 = np.dot(A.flatten(), B.flatten())
print(f"方法3: np.dot(A.flatten(), B.flatten()) = {frobenius_inner_3}")

# 手計算の確認
manual = 1*5 + 2*6 + 3*7 + 4*8
print(f"\n手計算: 1×5 + 2×6 + 3×7 + 4×8 = {manual}")
print(f"すべて一致: {frobenius_inner_1 == frobenius_inner_2 == frobenius_inner_3 == manual}")


行列 A:
[[1 2]
 [3 4]]

行列 B:
[[5 6]
 [7 8]]

方法1: np.sum(A * B) = 70
方法2: np.trace(A.T @ B) = 70
方法3: np.dot(A.flatten(), B.flatten()) = 70

手計算: 1×5 + 2×6 + 3×7 + 4×8 = 70
すべて一致: True


---

## 数学的な定義

### 1. Frobenius内積（標準的な定義）

$m \times n$ 行列 $A$ と $B$ のFrobenius内積：

$$\langle A, B \rangle_F = \sum_{i=1}^{m} \sum_{j=1}^{n} A_{ij} B_{ij}$$

### 2. トレースによる表現

$$\langle A, B \rangle_F = \text{tr}(A^T B) = \text{tr}(B^T A) = \text{tr}(AB^T) = \text{tr}(BA^T)$$

### 3. ベクトル化による表現

行列をベクトルに変換（列ベクトルを順に並べる）してから内積を取る：

$$\text{vec}(A) = \begin{pmatrix} A_{11} \\ A_{21} \\ \vdots \\ A_{m1} \\ A_{12} \\ \vdots \\ A_{mn} \end{pmatrix}$$

$$\langle A, B \rangle_F = \text{vec}(A)^T \text{vec}(B)$$

### 4. 例：3×2行列

$$A = \begin{pmatrix} 1 & 2 \\ 3 & 4 \\ 5 & 6 \end{pmatrix}, \quad B = \begin{pmatrix} 7 & 8 \\ 9 & 10 \\ 11 & 12 \end{pmatrix}$$

$$\langle A, B \rangle_F = 1 \times 7 + 3 \times 9 + 5 \times 11 + 2 \times 8 + 4 \times 10 + 6 \times 12$$
$$= 7 + 27 + 55 + 16 + 40 + 72 = 217$$


In [3]:
# 3×2行列の例
A = np.array([[1, 2],
              [3, 4],
              [5, 6]])
B = np.array([[7, 8],
              [9, 10],
              [11, 12]])

print("行列 A (3×2):")
print(A)
print("\n行列 B (3×2):")
print(B)

# Frobenius内積
inner_product = np.sum(A * B)
print(f"\nFrobenius内積 <A, B>_F = {inner_product}")

# 手計算の確認
print("\n手計算:")
print("1×7 + 3×9 + 5×11 + 2×8 + 4×10 + 6×12")
print(f"= {1*7} + {3*9} + {5*11} + {2*8} + {4*10} + {6*12}")
print(f"= {1*7 + 3*9 + 5*11 + 2*8 + 4*10 + 6*12}")

# ベクトル化による確認
A_vec = A.flatten()
B_vec = B.flatten()
print(f"\nベクトル化: vec(A) = {A_vec}")
print(f"ベクトル化: vec(B) = {B_vec}")
print(f"vec(A)^T vec(B) = {np.dot(A_vec, B_vec)}")


行列 A (3×2):
[[1 2]
 [3 4]
 [5 6]]

行列 B (3×2):
[[ 7  8]
 [ 9 10]
 [11 12]]

Frobenius内積 <A, B>_F = 217

手計算:
1×7 + 3×9 + 5×11 + 2×8 + 4×10 + 6×12
= 7 + 27 + 55 + 16 + 40 + 72
= 217

ベクトル化: vec(A) = [1 2 3 4 5 6]
ベクトル化: vec(B) = [ 7  8  9 10 11 12]
vec(A)^T vec(B) = 217


---

## 行列の内積の性質

Frobenius内積には以下の重要な性質があります：

### 1. 対称性（可換性）

$$\langle A, B \rangle_F = \langle B, A \rangle_F$$

### 2. 双線形性

- **加法**: $\langle A + B, C \rangle_F = \langle A, C \rangle_F + \langle B, C \rangle_F$
- **スカラー倍**: $\langle kA, B \rangle_F = k\langle A, B \rangle_F = \langle A, kB \rangle_F$

ここで、$k$ はスカラーです。

### 3. 正定値性

$$\langle A, A \rangle_F \geq 0$$

等号は $A = 0$（ゼロ行列）のときのみ成り立ちます。

### 4. 自分自身との内積とFrobeniusノルム

$$\|A\|_F = \sqrt{\langle A, A \rangle_F} = \sqrt{\sum_{i,j} A_{ij}^2}$$

これは**Frobeniusノルム（Frobenius norm）**と呼ばれます。

### 5. ゼロ行列との内積

$$\langle A, 0 \rangle_F = 0$$

任意の行列とゼロ行列の内積は0です。


In [4]:
# Frobenius内積の関数を定義
def frobenius_inner_product(A, B):
    """Frobenius内積を計算"""
    return np.sum(A * B)

# テスト用の行列
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])
C = np.array([[9, 10], [11, 12]])
k = 2

print("1. 対称性（可換性）")
inner_AB = frobenius_inner_product(A, B)
inner_BA = frobenius_inner_product(B, A)
print(f"<A, B>_F = {inner_AB}")
print(f"<B, A>_F = {inner_BA}")
print(f"一致: {np.allclose(inner_AB, inner_BA)}")

print("\n2. 双線形性 - 加法")
inner_AplusB_C = frobenius_inner_product(A + B, C)
inner_AC_plus_BC = frobenius_inner_product(A, C) + frobenius_inner_product(B, C)
print(f"<A+B, C>_F = {inner_AplusB_C}")
print(f"<A, C>_F + <B, C>_F = {inner_AC_plus_BC}")
print(f"一致: {np.allclose(inner_AplusB_C, inner_AC_plus_BC)}")

print("\n3. 双線形性 - スカラー倍")
inner_kA_B = frobenius_inner_product(k * A, B)
k_times_inner_AB = k * frobenius_inner_product(A, B)
print(f"<kA, B>_F = {inner_kA_B}")
print(f"k<A, B>_F = {k_times_inner_AB}")
print(f"一致: {np.allclose(inner_kA_B, k_times_inner_AB)}")

print("\n4. 正定値性とFrobeniusノルム")
inner_AA = frobenius_inner_product(A, A)
frobenius_norm_sq = np.sum(A ** 2)
frobenius_norm = np.linalg.norm(A, 'fro')
print(f"<A, A>_F = {inner_AA}")
print(f"||A||_F^2 = {frobenius_norm_sq}")
print(f"||A||_F = {frobenius_norm}")
print(f"一致: {np.allclose(inner_AA, frobenius_norm_sq)}")
print(f"正定値性: <A, A>_F >= 0 は {inner_AA >= 0}")

print("\n5. ゼロ行列との内積")
zero_matrix = np.zeros_like(A)
inner_A_zero = frobenius_inner_product(A, zero_matrix)
print(f"<A, 0>_F = {inner_A_zero}")


1. 対称性（可換性）
<A, B>_F = 70
<B, A>_F = 70
一致: True

2. 双線形性 - 加法
<A+B, C>_F = 388
<A, C>_F + <B, C>_F = 388
一致: True

3. 双線形性 - スカラー倍
<kA, B>_F = 140
k<A, B>_F = 140
一致: True

4. 正定値性とFrobeniusノルム
<A, A>_F = 30
||A||_F^2 = 30
||A||_F = 5.477225575051661
一致: True
正定値性: <A, A>_F >= 0 は True

5. ゼロ行列との内積
<A, 0>_F = 0


---

## 実装例

### 例1: 行列の類似度測定

2つの行列がどの程度似ているかをFrobenius内積を使って測定します。


In [5]:
def matrix_similarity(A, B):
    """Frobenius内積を使った行列の類似度を計算"""
    # 正規化されたFrobenius内積（コサイン類似度の行列版）
    norm_A = np.linalg.norm(A, 'fro')
    norm_B = np.linalg.norm(B, 'fro')

    if norm_A == 0 or norm_B == 0:
        return 0

    inner_product = frobenius_inner_product(A, B)
    similarity = inner_product / (norm_A * norm_B)

    return similarity, inner_product

# 例1: 似た行列
A1 = np.array([[1, 2], [3, 4]])
B1 = np.array([[1.1, 2.1], [3.1, 4.1]])  # A1に近い

# 例2: 異なる行列
A2 = np.array([[1, 2], [3, 4]])
B2 = np.array([[10, 20], [30, 40]])  # A2の10倍（方向は同じ）

# 例3: 直交的な行列
A3 = np.array([[1, 0], [0, 1]])  # 単位行列
B3 = np.array([[0, 1], [-1, 0]])  # 回転行列

print("例1: 似た行列")
sim1, inner1 = matrix_similarity(A1, B1)
print(f"A1 = \n{A1}")
print(f"B1 = \n{B1}")
print(f"Frobenius内積 = {inner1:.4f}")
print(f"類似度（正規化） = {sim1:.4f}")

print("\n例2: スカラー倍の関係")
sim2, inner2 = matrix_similarity(A2, B2)
print(f"A2 = \n{A2}")
print(f"B2 = \n{B2}")
print(f"Frobenius内積 = {inner2:.4f}")
print(f"類似度（正規化） = {sim2:.4f} (完全に同じ方向)")

print("\n例3: 直交的な行列")
sim3, inner3 = matrix_similarity(A3, B3)
print(f"A3 = \n{A3}")
print(f"B3 = \n{B3}")
print(f"Frobenius内積 = {inner3:.4f}")
print(f"類似度（正規化） = {sim3:.4f} (直交)")


例1: 似た行列
A1 = 
[[1 2]
 [3 4]]
B1 = 
[[1.1 2.1]
 [3.1 4.1]]
Frobenius内積 = 31.0000
類似度（正規化） = 0.9999

例2: スカラー倍の関係
A2 = 
[[1 2]
 [3 4]]
B2 = 
[[10 20]
 [30 40]]
Frobenius内積 = 300.0000
類似度（正規化） = 1.0000 (完全に同じ方向)

例3: 直交的な行列
A3 = 
[[1 0]
 [0 1]]
B3 = 
[[ 0  1]
 [-1  0]]
Frobenius内積 = 0.0000
類似度（正規化） = 0.0000 (直交)


### 例2: 行列の直交性

2つの行列が「直交」しているかどうかを判定します。行列が直交するとは、Frobenius内積が0になることです。


In [6]:
def are_matrices_orthogonal(A, B, tolerance=1e-10):
    """2つの行列が直交しているかどうかを判定"""
    inner_product = frobenius_inner_product(A, B)
    return abs(inner_product) < tolerance

# 例1: 直交する行列
A1 = np.array([[1, 0], [0, 1]])  # 単位行列
B1 = np.array([[0, 1], [-1, 0]])  # 回転行列

print("例1: 直交する行列")
print(f"A1 = \n{A1}")
print(f"B1 = \n{B1}")
print(f"Frobenius内積 = {frobenius_inner_product(A1, B1)}")
print(f"直交: {are_matrices_orthogonal(A1, B1)}")

# 例2: 直交しない行列
A2 = np.array([[1, 2], [3, 4]])
B2 = np.array([[5, 6], [7, 8]])

print("\n例2: 直交しない行列")
print(f"A2 = \n{A2}")
print(f"B2 = \n{B2}")
print(f"Frobenius内積 = {frobenius_inner_product(A2, B2)}")
print(f"直交: {are_matrices_orthogonal(A2, B2)}")

# 例3: より複雑な直交行列の例
A3 = np.array([[1, 1], [1, -1]])
B3 = np.array([[1, -1], [-1, -1]])

print("\n例3: 別の直交行列の例")
print(f"A3 = \n{A3}")
print(f"B3 = \n{B3}")
print(f"Frobenius内積 = {frobenius_inner_product(A3, B3)}")
print(f"直交: {are_matrices_orthogonal(A3, B3)}")


例1: 直交する行列
A1 = 
[[1 0]
 [0 1]]
B1 = 
[[ 0  1]
 [-1  0]]
Frobenius内積 = 0
直交: True

例2: 直交しない行列
A2 = 
[[1 2]
 [3 4]]
B2 = 
[[5 6]
 [7 8]]
Frobenius内積 = 70
直交: False

例3: 別の直交行列の例
A3 = 
[[ 1  1]
 [ 1 -1]]
B3 = 
[[ 1 -1]
 [-1 -1]]
Frobenius内積 = 0
直交: True


### 例3: 異なるサイズの行列の内積

異なるサイズの行列の内積を計算する方法を説明します。


In [7]:
# 注意: Frobenius内積は同じサイズの行列に対してのみ定義されます
# 異なるサイズの場合は、適切に処理する必要があります

# 例: 2×3行列と3×2行列
A = np.array([[1, 2, 3],
                [4, 5, 6]])  # 2×3

B = np.array([[7, 8],
              [9, 10],
              [11, 12]])  # 3×2

print("行列 A (2×3):")
print(A)
print("\n行列 B (3×2):")
print(B)

# 方法1: A と B^T のFrobenius内積
# A と B^T は同じサイズ (2×3)
BT = B.T
print(f"\nB^T (2×3):")
print(BT)
inner_ABT = frobenius_inner_product(A, BT)
print(f"<A, B^T>_F = {inner_ABT}")

# 方法2: A^T と B のFrobenius内積
# A^T と B は同じサイズ (3×2)
AT = A.T
print(f"\nA^T (3×2):")
print(AT)
inner_ATB = frobenius_inner_product(AT, B)
print(f"<A^T, B>_F = {inner_ATB}")

# 方法3: ベクトル化してから内積（サイズが異なる場合は注意）
# この場合、要素数が同じでないと内積は定義されない
print(f"\nAの要素数: {A.size}, Bの要素数: {B.size}")
if A.size == B.size:
    inner_vec = np.dot(A.flatten(), B.flatten())
    print(f"ベクトル化した内積 = {inner_vec}")
else:
    print("要素数が異なるため、ベクトル化した内積は定義されません")

# 実際には、AB や BA のトレースを使うこともある
print(f"\nAB (2×2):")
AB = A @ B
print(AB)
print(f"tr(AB) = {np.trace(AB)}")

print(f"\nBA (3×3):")
BA = B @ A
print(BA)
print(f"tr(BA) = {np.trace(BA)}")


行列 A (2×3):
[[1 2 3]
 [4 5 6]]

行列 B (3×2):
[[ 7  8]
 [ 9 10]
 [11 12]]

B^T (2×3):
[[ 7  9 11]
 [ 8 10 12]]
<A, B^T>_F = 212

A^T (3×2):
[[1 4]
 [2 5]
 [3 6]]
<A^T, B>_F = 212

Aの要素数: 6, Bの要素数: 6
ベクトル化した内積 = 217

AB (2×2):
[[ 58  64]
 [139 154]]
tr(AB) = 212

BA (3×3):
[[ 39  54  69]
 [ 49  68  87]
 [ 59  82 105]]
tr(BA) = 212


---

## ベクトル内積との関係

### 1. ベクトルは1次元配列として扱える

ベクトル $\mathbf{a} \in \mathbb{R}^n$ は $n \times 1$ または $1 \times n$ の行列として扱えます。

### 2. 列ベクトルの内積

列ベクトル $\mathbf{a}, \mathbf{b} \in \mathbb{R}^n$ の場合：

$$\mathbf{a} \cdot \mathbf{b} = \mathbf{a}^T \mathbf{b} = \text{tr}(\mathbf{a} \mathbf{b}^T) = \langle \mathbf{a} \mathbf{b}^T, I \rangle_F$$

### 3. 行ベクトルの内積

行ベクトルの場合も同様に、Frobenius内積として表現できます。


In [8]:
# ベクトル内積と行列内積の関係

# 列ベクトルとして扱う
a = np.array([[1], [2], [3]])  # 3×1行列
b = np.array([[4], [5], [6]])  # 3×1行列

print("列ベクトル a (3×1):")
print(a)
print("\n列ベクトル b (3×1):")
print(b)

# 方法1: 通常のベクトル内積（転置してから積）
inner_1 = (a.T @ b)[0, 0]
print(f"\n方法1: a^T b = {inner_1}")

# 方法2: Frobenius内積として計算
# a と b は同じサイズなので直接計算できない
# 代わりに、外積 ab^T のトレースを使う
abT = a @ b.T
print(f"\nab^T (3×3):")
print(abT)
inner_2 = np.trace(abT)
print(f"方法2: tr(ab^T) = {inner_2}")

# 方法3: ベクトル化（1次元配列として扱う）
a_flat = a.flatten()
b_flat = b.flatten()
inner_3 = np.dot(a_flat, b_flat)
print(f"\n方法3: ベクトル化した内積 = {inner_3}")

print(f"\nすべて一致: {inner_1 == inner_2 == inner_3}")

# 通常のNumPyベクトル（1次元配列）との比較
a_vec = np.array([1, 2, 3])
b_vec = np.array([4, 5, 6])
inner_vec = np.dot(a_vec, b_vec)
print(f"\nNumPyベクトル（1次元）の内積: {inner_vec}")
print(f"一致: {inner_vec == inner_1}")


列ベクトル a (3×1):
[[1]
 [2]
 [3]]

列ベクトル b (3×1):
[[4]
 [5]
 [6]]

方法1: a^T b = 32

ab^T (3×3):
[[ 4  5  6]
 [ 8 10 12]
 [12 15 18]]
方法2: tr(ab^T) = 32

方法3: ベクトル化した内積 = 32

すべて一致: True

NumPyベクトル（1次元）の内積: 32
一致: True


---

## 機械学習での応用例

### 1. ニューラルネットワークの重み行列の正則化

重み行列のFrobeniusノルム（自分自身との内積の平方根）を正則化項として使用します：

$$L = \text{Loss} + \lambda \|W\|_F^2 = \text{Loss} + \lambda \langle W, W \rangle_F$$


In [9]:
# 重み行列の正則化の例
def l2_regularization_loss(weights, lambda_reg=0.01):
    """L2正則化項を計算（Frobeniusノルムの2乗）"""
    # ||W||_F^2 = <W, W>_F
    frobenius_norm_sq = frobenius_inner_product(weights, weights)
    return lambda_reg * frobenius_norm_sq

# 例: ニューラルネットワークの重み行列
W1 = np.random.randn(10, 5)  # 入力層から隠れ層への重み
W2 = np.random.randn(5, 3)   # 隠れ層から出力層への重み

print("重み行列 W1 (10×5):")
print(f"Frobeniusノルム: {np.linalg.norm(W1, 'fro'):.4f}")
print(f"Frobeniusノルムの2乗: {frobenius_inner_product(W1, W1):.4f}")

print("\n重み行列 W2 (5×3):")
print(f"Frobeniusノルム: {np.linalg.norm(W2, 'fro'):.4f}")
print(f"Frobeniusノルムの2乗: {frobenius_inner_product(W2, W2):.4f}")

# 正則化項
lambda_reg = 0.01
reg_loss = l2_regularization_loss(W1, lambda_reg) + l2_regularization_loss(W2, lambda_reg)
print(f"\nL2正則化項 (λ={lambda_reg}): {reg_loss:.4f}")


重み行列 W1 (10×5):
Frobeniusノルム: 7.3757
Frobeniusノルムの2乗: 54.4006

重み行列 W2 (5×3):
Frobeniusノルム: 3.9060
Frobeniusノルムの2乗: 15.2570

L2正則化項 (λ=0.01): 0.6966


### 2. 行列分解（Matrix Factorization）

推薦システムなどで、ユーザー行列とアイテム行列の内積を使って評価を予測します。


In [10]:
# 行列分解の例（簡易版）
# ユーザー×アイテムの評価行列を、ユーザー特徴行列とアイテム特徴行列に分解

# 元の評価行列（一部が欠損）
R = np.array([[5, 3, 0, 1],
              [4, 0, 0, 1],
              [1, 1, 0, 5],
              [1, 0, 0, 4],
              [0, 1, 5, 4]])

print("評価行列 R (5×4):")
print(R)

# 低ランク近似: R ≈ U V^T
# U: ユーザー特徴行列 (5×2)
# V: アイテム特徴行列 (4×2)
rank = 2
U = np.random.randn(5, rank) * 0.5
V = np.random.randn(4, rank) * 0.5

print(f"\nユーザー特徴行列 U (5×{rank}):")
print(U)
print(f"\nアイテム特徴行列 V (4×{rank}):")
print(V)

# 予測評価行列: R_pred = U V^T
R_pred = U @ V.T
print(f"\n予測評価行列 R_pred = U V^T (5×4):")
print(R_pred)

# 各要素は、ユーザー特徴ベクトルとアイテム特徴ベクトルの内積
# R_pred[i, j] = U[i, :] @ V[j, :]
print("\n例: ユーザー0とアイテム0の評価予測")
print(f"U[0, :] = {U[0, :]}")
print(f"V[0, :] = {V[0, :]}")
print(f"内積 = {np.dot(U[0, :], V[0, :]):.4f}")
print(f"R_pred[0, 0] = {R_pred[0, 0]:.4f}")


評価行列 R (5×4):
[[5 3 0 1]
 [4 0 0 1]
 [1 1 0 5]
 [1 0 0 4]
 [0 1 5 4]]

ユーザー特徴行列 U (5×2):
[[-0.05454193 -0.10273155]
 [-0.66807116  0.12693034]
 [-0.87419915  0.40314271]
 [ 0.21680377  0.08339824]
 [-1.06506976  1.17048701]]

アイテム特徴行列 V (4×2):
[[ 0.45968592 -0.06785518]
 [ 1.04772952  0.33044272]
 [ 0.11081521  0.08073276]
 [ 0.53018802  0.52356069]]

予測評価行列 R_pred = U V^T (5×4):
[[-0.01810129 -0.09109208 -0.01433788 -0.08270368]
 [-0.31571579 -0.65801467 -0.06378501 -0.28774759]
 [-0.42921236 -0.78270868 -0.06432774 -0.25242023]
 [ 0.09400264  0.25471005  0.03075813  0.1586108 ]
 [-0.56902118 -0.72912612 -0.02352928  0.04813377]]

例: ユーザー0とアイテム0の評価予測
U[0, :] = [-0.05454193 -0.10273155]
V[0, :] = [ 0.45968592 -0.06785518]
内積 = -0.0181
R_pred[0, 0] = -0.0181


### 3. 主成分分析（PCA）での共分散行列

PCAでは、データ行列の共分散行列（内積の行列）の固有ベクトルを計算します。


In [11]:
# PCAの例（簡易版）
# データ行列 X (n_samples × n_features)
np.random.seed(42)
X = np.random.randn(100, 3)  # 100サンプル、3特徴量

# データを中心化（平均を0に）
X_centered = X - X.mean(axis=0)

print("データ行列 X (100×3):")
print(f"形状: {X.shape}")
print(f"最初の5行:\n{X[:5]}")

# 共分散行列: C = (1/(n-1)) X^T X
# X^T X は各特徴量間の内積の行列
XTX = X_centered.T @ X_centered
cov_matrix = XTX / (X.shape[0] - 1)

print(f"\nX^T X (内積の行列, 3×3):")
print(XTX)
print(f"\n共分散行列 C (3×3):")
print(cov_matrix)

# 固有値分解
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

print(f"\n固有値（降順）: {eigenvalues[::-1]}")
print(f"\n第1主成分（最大固有値に対応）: {eigenvectors[:, -1]}")

# 主成分への射影は、データベクトルと主成分ベクトルの内積
first_pc = eigenvectors[:, -1]
projection = X_centered @ first_pc
print(f"\n第1主成分への射影（最初の5サンプル）: {projection[:5]}")


データ行列 X (100×3):
形状: (100, 3)
最初の5行:
[[ 0.49671415 -0.1382643   0.64768854]
 [ 1.52302986 -0.23415337 -0.23413696]
 [ 1.57921282  0.76743473 -0.46947439]
 [ 0.54256004 -0.46341769 -0.46572975]
 [ 0.24196227 -1.91328024 -1.72491783]]

X^T X (内積の行列, 3×3):
[[ 67.34905879  -3.88836444 -10.96925958]
 [ -3.88836444  94.90505264 -13.35108063]
 [-10.96925958 -13.35108063 122.6182803 ]]

共分散行列 C (3×3):
[[ 0.68029352 -0.03927641 -0.1108006 ]
 [-0.03927641  0.9586369  -0.1348594 ]
 [-0.1108006  -0.1348594   1.23856849]]

固有値（降順）: [1.30567458 0.92688528 0.64493904]

第1主成分（最大固有値に対応）: [-0.14276678 -0.34442383  0.9278954 ]

第1主成分への射影（最初の5サンプル）: [ 0.458259   -0.47348012 -1.04484052 -0.46943141 -1.09554364]


### 4. 勾配降下法での更新式

最適化問題で、勾配行列と更新行列の内積を使って損失を計算します。

---

## まとめ

行列の内積（Frobenius内積）は、線形代数と機械学習の重要な概念です：

### 重要なポイント

1. **定義**: $\langle A, B \rangle_F = \sum_{i,j} A_{ij} B_{ij} = \text{tr}(A^T B)$

2. **性質**:
   - 対称性: $\langle A, B \rangle_F = \langle B, A \rangle_F$
   - 双線形性: 加法とスカラー倍について線形
   - 正定値性: $\langle A, A \rangle_F \geq 0$

3. **Frobeniusノルム**: $\|A\|_F = \sqrt{\langle A, A \rangle_F}$

4. **応用**:
   - 行列の類似度測定
   - 正則化（L2正則化）
   - 行列分解
   - 最適化問題

### ベクトル内積との関係

- ベクトルは行列の特殊な場合
- 列ベクトルの内積: $\mathbf{a}^T \mathbf{b} = \text{tr}(\mathbf{a} \mathbf{b}^T)$
- 行列の内積はベクトル内積の自然な拡張

行列の内積を理解することで、機械学習の多くのアルゴリズムをより深く理解できるようになります。
